In [2]:
import numpy as np


class Decoder(object):
    def __init__(self, initialProb, transProb, obsProb):
        self.N = initialProb.shape[0]
        self.initialProb = initialProb
        self.transProb = transProb
        self.obsProb = obsProb
        assert self.initialProb.shape == (self.N, 1)
        assert self.transProb.shape == (self.N, self.N)
        assert self.obsProb.shape[0] == self.N
        
    def Obs(self, obs):
        return self.obsProb[:, obs, None]

    def Decode(self, obs):
        trellis = np.zeros((self.N, len(obs)))
        backpt = np.ones((self.N, len(obs)), 'int32') * -1
                
        # initialization
        trellis[:, 0] = np.squeeze(self.initialProb * self.Obs(obs[0]))
                
        for t in xrange(1, len(obs)):
            trellis[:, t] = (trellis[:, t-1, None].dot(self.Obs(obs[t]).T) * self.transProb).max(0)
            backpt[:, t] = (np.tile(trellis[:, t-1, None], [1, self.N]) * self.transProb).argmax(0)
        # termination
        tokens = [trellis[:, -1].argmax()]
        for i in xrange(len(obs)-1, 0, -1):
            tokens.append(backpt[tokens[-1], i])
        return tokens[::-1]

In [3]:
import pandas
from pandas import DataFrame
import nltk
infile="hmm.xlsx"

In [4]:
Apandas = pandas.read_excel(infile,sheetname="A") 
rownames = Apandas.index.tolist()
A=np.array(Apandas)

In [5]:
Bpandas = pandas.read_excel(infile,'B')
B=np.array(Bpandas)
statenames = Bpandas.index.tolist()

In [6]:
piPandas= pandas.read_excel(infile,'pi')
pi=np.array(piPandas)

In [7]:
decoder = Decoder(pi,B,A)
states =  decoder.Decode(np.arange(5))
result = np.array(statenames)[states].tolist()
sentence = Apandas.columns.tolist()
resultTagged = zip(sentence,result)

In [8]:
result

[u'NNP', u'MD', u'VB', u'DT', u'NN']

In [11]:
Apandas.shape

(7, 5)

In [12]:
Bpandas.shape

(7, 7)